In [1]:
from thermo import PRMIX, PR78MIX, SRKMIX

import dipypr as dp

import numpy as np

Tc = [320, 375]
Pc = [30*1e5, 45*1e5]
w = [0.0123, 0.045]

n = [4.0, 6.0]
z  = [0.4, 0.6]
V = 1.0 # m^3/mol
T = 298.15

In [2]:
names = ['pentane', 'hexane']

chms = [dp.Chemical(n) for n in names]
tc = [chm.cons.critical_temperature.magnitude for chm in chms]
pc = [chm.cons.critical_pressure.to("Pa").magnitude for chm in chms]
w = [chm.cons.acentric_factor.magnitude for chm in chms]
vc = [chm.cons.critical_volume.to("L/mol").magnitude for chm in chms]

In [3]:
tc

[469.7, 507.6]

In [4]:
pc

[3370000.0, 3025000.0]

In [5]:
w

[0.251506, 0.301261]

In [6]:
m_names = ["PR76", "PR78", "SRK"]

mods = [PRMIX, PR78MIX, SRKMIX]

models = []

for mod in mods:
    models.append(mod(Tcs=tc, Pcs=pc, omegas=w, zs=z, T=T, V=V))

In [7]:
coso = PRMIX(Tcs=tc, Pcs=pc, omegas=w, zs=z, T=T, V=V)

# Pressure

In [8]:
p = []
dpdv = []
dpdt = []
dpdn = []

for i in range(3):
    p.append(models[i].P / 1e5)  # Convert Pa to bar
    dpdv.append(models[i].dP_dV_g / 1e5 / 1000)
    dpdt.append(models[i].dP_dT_g / 1e5)
    dpdn.append((np.array(models[i].dP_dns_Vt("g")) / 1e5).tolist())
    
print("p_exp =", p)
print("dpdv_exp =", dpdv)
print("dpdt_exp =", dpdt)
print("dpdn_exp =", dpdn)
    

p_exp = [0.024758433028831645, 0.024758433028831645, 0.024759435993050086]
dpdv_exp = [-2.472730282235202e-05, -2.472730282235202e-05, -2.472930571514075e-05]
dpdt_exp = [8.321187984959119e-05, 8.321187984959119e-05, 8.321898105202635e-05]
dpdn_exp = [[0.024733557386271844, 0.024723133113072143], [0.024733557386271844, 0.024723133113072143], [0.024735507908033087, 0.02472517091987919]]


# Residual enthalpy TV

In [9]:
h = []
dhdv = []
dhdt = []
dhdn = []

for i in range(3):
    h.append(models[i].H_dep_g / 100)
    dhdv.append(models[i].dH_dep_dV_g_T / 100 / 1000)
    dhdt.append(models[i].dH_dep_dT_g_V / 100)
    dhdn.append((np.array(models[i].dnH_dep_dns(models[i].Z_g)) / 100).tolist())
    
print("h_exp =", h)
print("dhdv_exp =", dhdv)
print("dhdt_exp =", dhdt)
print("dhdn_exp =", dhdn)

h_exp = [-0.08233139437821592, -0.08233139437821592, -0.08244045745725088]
dhdv_exp = [8.231915480358669e-05, 8.231915480358669e-05, 8.243348552089947e-05]
dhdt_exp = [0.00011201776832480803, 0.00011201776832480803, 0.00012596855942014428]
dhdn_exp = [[-0.0659512052968063, -0.093251520432489], [-0.0659512052968063, -0.093251520432489], [-0.06580850526451794, -0.09352842558573954]]


# Residual entropy VT

In [18]:
s = []
dsdv = []
dsdt = []

R = 0.08314462618

for i in range(3):
    s.append((models[i].S_dep_g / 100 - R * np.log(models[i].Z_g)).tolist())
    dsdv.append(models[i].dS_dep_dV_g_T / 100 / 1000 / 10 - R * (V / R / T * dpdv[i] / 10 + p[i] / R / T) / models[i].Z_g)
    dsdt.append(models[i].dS_dep_dT_g_V / 100)
    
print("s_exp =", s)
print("dsdv_exp =", dsdv)
print("dsdt_exp =", dsdt)

s_exp = [-6.725919527969187e-05, -6.725919527969187e-05, -7.435800385866321e-05]
dsdv_exp = [-8.311914255274389e-05, -8.311914255274389e-05, -8.311876830564422e-05]
dsdt_exp = [7.26710035661321e-07, 7.26710035661321e-07, 7.622103462157431e-07]


# Residual Gibbs VT

In [23]:
g = []
dgdv = []
dgdt = []

R = 0.08314462618

for i in range(3):
    g.append((models[i].G_dep_g / 100 + R * T * np.log(models[i].Z_g)).tolist())
    
print("g_exp =", g)

g_exp = [-0.06227806530557579, -0.06227806530557579, -0.06027061860679045]


# Cp

In [27]:
cp = []

for i in range(3):
    cp.append(models[i].Cp_dep_g / 100)
    
print("cp_exp = ", cp)

cp_exp =  [0.0003890367245792703, 0.0003890367245792703, 0.0004033734630853303]


# Cv

In [28]:
cv = []

for i in range(3):
    cv.append(models[i].Cv_dep_g / 100)
    
print("cv_exp = ", cv)

cv_exp =  [4.476410026600885e-05, 4.476410026600885e-05, 5.161368892615888e-05]
